# Для тренировки модели нужно установить turicreate
Linux or mac
python 3.6 - 3.8

`pip install turicreate`

### На вход:
- _attend_features.csv (с нашими фичами)
- _user_features.csv (с нашими фичами)
- _groups.csv

### Выход:
- predicts/predict_v01.csv


In [1]:
import turicreate as tc
import pandas as pd

In [2]:
actions = tc.SFrame.read_csv('_attend_features.csv')

Finished parsing file /media/vasiliev/SSD_Data/_dev/ezee_may_2003/ds/_attend_features.csv

Parsing completed. Parsed 100 lines in 0.195318 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,float,str,str,float,str,str,str,str,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 219940 lines. Lines per second: 888496

Finished parsing file /media/vasiliev/SSD_Data/_dev/ezee_may_2003/ds/_attend_features.csv

Parsing completed. Parsed 600390 lines in 0.53239 secs.

In [3]:
#items = tc.SFrame.read_csv('groups.csv')

There are a variety of machine learning techniques that can be used to build a recommender model. Turi Create provides a method turicreate.recommender.create that will automatically choose an appropriate model for your data set. More details on choosing a model: https://apple.github.io/turicreate/docs/userguide/recommender/choosing-a-model.html

First we create a random split of the data to produce a validation set that can be used to evaluate the model.

### Чему учим модель

Мы учим модель предсказывать логарифм от количества посещений юзером курса (логарифм потому что разброс посещений курса пользователем высокий - от 1 до 64)

### Создание и обучение дефолтной модели

In [4]:
training_data, validation_data = tc.recommender.util.random_split_by_user(actions, 'user_id', 'group_id')
model = tc.recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits')

Preparing data set.

Data has 597961 observations with 50881 users and 24534 items.

Data prepared in: 0.385187s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.141846

Final training RMSE: 0.347505

In [5]:
# параметры модели
model

Class                            : RankingFactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : group_id
Target                           : log_visits
Additional observation features  : 10
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 597961
Number of users                  : 50881
Number of items                  : 24534

Training summary
----------------
Training time                    : 11.3381

Model Parameters
----------------
Model class                      : RankingFactorizationRecommender
num_factors                      : 32
binary_target                    : 0
side_data_factorization          : 1
solver                           : auto
nmf                              : 0
max_iterations                   : 25

Regularization Settings
-----------------------
regularization                   : 0.0
regularization_type          

### Получение предсказаний от дефолтной модели

In [6]:
default_recommendations = model.recommend()

recommendations finished on 1000/50881 queries. users per second: 6490.35

recommendations finished on 2000/50881 queries. users per second: 6572.01

recommendations finished on 3000/50881 queries. users per second: 6591.13

recommendations finished on 4000/50881 queries. users per second: 6599.46

recommendations finished on 5000/50881 queries. users per second: 6596.15

...

recommendations finished on 46000/50881 queries. users per second: 6658.7

recommendations finished on 47000/50881 queries. users per second: 6659.51

recommendations finished on 48000/50881 queries. users per second: 6660.88

recommendations finished on 49000/50881 queries. users per second: 6662.39

recommendations finished on 50000/50881 queries. users per second: 6662.93

### Посмотрим рекоммендации для отдельных юзеров

In [7]:
df_groups = pd.read_csv('groups.csv')

def display_history_and_predictions_for_user(user_id, train_dataset, predictions):
    print('----------------------------------------------------------------------------')
    print('User', user_id, 'history:')
    display(train_dataset[train_dataset['user_id'] == user_id][
        'group_id','is_online','session','направление 1','направление 2'])
    predictions = predictions[predictions['user_id'] == user_id].to_dataframe()
    print('User recommendations:')
    predicted_groups = predictions['group_id'].to_list()
    display(df_groups[df_groups['уникальный номер'].isin(predicted_groups)].drop(columns=[
        'адрес площадки','расписание в активных периодах','расписание в закрытых периодах','расписание в плановом периоде']))

In [8]:
user_id = 101352882

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101352882 history:


group_id,is_online,session,направление 1,направление 2
801348234,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801348838,offline,mornig,Физическая активность,Гимнастика
801348944,offline,mornig,Танцы,Танцы
801349048,offline,mornig,Физическая активность,ОФП
801350227,online,mornig,Творчество,ОНЛАЙН Художественно-прикладное творчество ...
801350380,online,mornig,Танцы,ОНЛАЙН Танцы
801351413,online,mornig,Рисование,ОНЛАЙН Рисование
801358464,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801358465,online,mornig,Танцы,ОНЛАЙН Танцы
801358466,online,mornig,Физическая активность,ОНЛАЙН Гимнастика


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
1391,801350123,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северный административный округ,муниципальный округ Хорошевский
2022,801350665,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
2040,801356727,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
2047,801350944,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
2287,801350038,Образование,ОНЛАЙН Образовательный практикум,ОНЛАЙН География. Путешествия вокруг света,NaN,NaN
3423,801351614,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN
3761,801350894,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
3973,801350683,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN
5404,801349506,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии по Москв...,Северный административный округ,муниципальный округ Савеловский
7838,801351772,Образование,"ОНЛАЙН История, искусство, краеведение",ОНЛАЙН История искусства,Северный административный округ,муниципальный округ Коптево


In [9]:
user_id = 101354253

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101354253 history:


group_id,is_online,session,направление 1,направление 2
801347802,online,mornig,Физическая активность,ОНЛАЙН ОФП
801348667,online,evening,Физическая активность,ОНЛАЙН Гимнастика
801348794,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801349796,offline,mornig,Физическая активность,Гимнастика
801349875,offline,aftennoon,Физическая активность,Гимнастика
801350556,online,mornig,Физическая активность,ОНЛАЙН ОФП
801350568,online,mornig,Физическая активность,ОНЛАЙН ОФП
801361065,offline,mornig,Рисование,Рисование


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
517,801347156,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Здоровая спина,"Восточный административный округ, Восточный ад...","муниципальный округ Вешняки, муниципальный окр..."
3324,801352656,Творчество,ОНЛАЙН Красота и стиль,ОНЛАЙН Мастер-класс по уходу за кожей в зрелом...,NaN,NaN
3332,801352699,Творчество,ОНЛАЙН Красота и стиль,ОНЛАЙН Мастер-класс по уходу за кожей в зрелом...,NaN,NaN
3649,801349265,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Дыхательная гимнастика,NaN,NaN
3743,801350414,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Архитектура Москвы и Подмосковья,NaN,NaN
3769,801353869,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
3775,801352905,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
3777,801350706,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
11464,801352004,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN
12287,801347321,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН Лечебная физкультура,Восточный административный округ,муниципальный округ Восточный


In [10]:
user_id = 101385865

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101385865 history:


group_id,is_online,session,направление 1,направление 2
801346923,online,mornig,Танцы,ОНЛАЙН Танцы
801348309,online,mornig,Творчество,ОНЛАЙН Красота и стиль
801348946,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801349093,online,mornig,Образование,ОНЛАЙН Иностранные языки
801350488,offline,aftennoon,Физическая активность,"Фитнес, тренажеры"
801350739,offline,mornig,Танцы,Танцы
801350759,offline,mornig,Танцы,Танцы
801351106,online,mornig,Творчество,ОНЛАЙН Красота и стиль
801351519,offline,mornig,Танцы,Танцы
801351888,offline,mornig,Танцы,Танцы


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
2172,801351211,Образование,"ОНЛАЙН История, искусство, краеведение",ОНЛАЙН История искусства,NaN,NaN
2184,801352464,Образование,"ОНЛАЙН История, искусство, краеведение",ОНЛАЙН История искусства,NaN,NaN
2191,801351432,Образование,"ОНЛАЙН История, искусство, краеведение",ОНЛАЙН Основы духовной культуры,NaN,NaN
2486,801351262,Творчество,ОНЛАЙН Красота и стиль,ОНЛАЙН История моды и прически,NaN,NaN
3307,801348819,Творчество,ОНЛАЙН Красота и стиль,ОНЛАЙН Мастер-класс по уходу за кожей в зрелом...,NaN,NaN
3383,801351733,Творчество,ОНЛАЙН Красота и стиль,ОНЛАЙН Мастер-класс по уходу за кожей в зрелом...,NaN,NaN
3743,801350414,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Архитектура Москвы и Подмосковья,NaN,NaN
3771,801350459,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
3775,801352905,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
4982,801350902,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии по Москв...,NaN,NaN


### Делаем сравнение различных моделей

Описание всех моделей, заложенных в turicreate, доступно по ссылке:

https://apple.github.io/turicreate/docs/api/turicreate.toolkits.recommender.html

In [11]:
model_0 = tc.recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits')
model_1 = tc.item_similarity_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits')
model_2 = tc.popularity_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits')
model_3 = tc.factorization_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits')
model_4 = tc.ranking_factorization_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits')

Preparing data set.

Data has 597961 observations with 50881 users and 24534 items.

Data prepared in: 0.370567s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.134109

Final training RMSE: 0.338663

In [12]:
print(tc.recommender.util.compare_models(
    validation_data, 
    [model_0, model_1,model_2,model_3,model_4], 
    model_names=["Default model",
                 "Item similarity model",
                 'Popularity recommender',
                 'Factorization recommender',
                 'Ranking factorization recommender']))

PROGRESS: Evaluate model Default model

Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0013888888888888892 | 0.0001984126984126984 |
|   2    | 0.0013888888888888892 | 0.0015873015873015875 |
|   3    | 0.0013888888888888872 | 0.0019345238095238094 |
|   4    |  0.001388888888888889 | 0.0022123015873015887 |
|   5    |  0.001388888888888889 |  0.002247914122914122 |
|   6    | 0.0011574074074074073 | 0.0022479141229141235 |
|   7    |  0.000992063492063492 |  0.002247914122914123 |
|   8    | 0.0010416666666666675 | 0.0036368030118030105 |
|   9    | 0.0009259259259259261 | 0.0036368030118030118 |
|   10   |  0.000972222222222222 | 0.0037029405779405783 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.2420882328252758

Per User RMSE (best)
+----

`Default model:             `Overall RMSE: 1.4839031983440507

`Item similarity model:     `Overall RMSE: 1.9387119289000498

`Popularity recommender:    `Overall RMSE: 0.9897593711404455

`Factorization recommender: `Overall RMSE: 1.0964792988739434

`Ranking factorization:     `Overall RMSE: 1.0114718154853628

### Берем результаты Ranking factorization модели

In [13]:
preds = model_4.recommend(k=10)
display(preds)

recommendations finished on 1000/50881 queries. users per second: 6468.18

recommendations finished on 2000/50881 queries. users per second: 6601.95

recommendations finished on 3000/50881 queries. users per second: 6654.94

recommendations finished on 4000/50881 queries. users per second: 6677.26

recommendations finished on 5000/50881 queries. users per second: 6694.35

...

recommendations finished on 47000/50881 queries. users per second: 6739.77

recommendations finished on 48000/50881 queries. users per second: 6740.36

recommendations finished on 49000/50881 queries. users per second: 6740.71

recommendations finished on 50000/50881 queries. users per second: 6741.91

user_id,group_id,score,rank
101370217,801352919,2.3939897183360683,1
101370217,801352970,2.3840604487838375,2
101370217,801349361,2.3687165740909206,3
101370217,801350828,2.2919596347989666,4
101370217,801355404,2.2737267974795925,5
101370217,801351513,2.214499360698663,6
101370217,801351817,2.2107914749564754,7
101370217,801353793,2.2099186990441906,8
101370217,801349440,2.2045488420905697,9
101370217,801346776,2.1763000849666225,10


### Сохраняем результаты для бэка

In [14]:
preds.to_dataframe().to_csv('predicts/predict_v01.csv', index=False )

### Делаем предсказание для Test

In [65]:
import json
with open('groups2023.json') as json_file:
    groups2023 = json.load(json_file)

df_test = pd.read_csv('test.csv')
df_test.head()
df_test['уникальный номер участника'] = df_test['уникальный номер участника'].astype(int)


def display_history_and_predictions_for_user2(user_id, train_dataset, predictions):
    print('----------------------------------------------------------------------------')
    print('User', user_id, 'history:')
    display(train_dataset[train_dataset['user_id'] == user_id][
        'group_id','is_online','session','направление 1','направление 2'].to_dataframe())
    predictions = predictions[predictions['user_id'] == user_id]
    print('User recommendations:')
    predicted_groups = predictions['group_id'].to_list()
    display(df_groups[df_groups['уникальный номер'].isin(predicted_groups)].drop(columns=[
        'адрес площадки',
        'расписание в активных периодах',
        'расписание в закрытых периодах',
        'расписание в плановом периоде']))

results = []  
for index, row in df_test.iterrows():
    # Получаем юзера
    user_id = row['уникальный номер участника']
    x = model.recommend(users=[user_id], k=1000).to_dataframe()
    x = x[x['group_id'].isin(groups2023)]
    #Выбираем первые 10 предсказаний
    x = x.head(10)

    groups_list = x['group_id'].to_list()
    groups_list = [str(x) for x in groups_list]    
    groups_str = ','.join(groups_list)
    results.append(groups_str)
    #print(groups_str)
    
df_test['уникальный номер группы'] = results
df_test.to_csv('test.xsv', index=False)


In [62]:
df_test.loc[0]['уникальный номер группы'] = 1

/tmp/ipykernel_4305/1640993157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.loc[0]['уникальный номер группы'] = 1


In [63]:
display(df_test)

,уникальный номер участника,уникальный номер группы
0,101346610,NaN
1,101346612,NaN
2,101347269,NaN
3,101347321,NaN
4,101347983,NaN
...,...,...
195,101447423,NaN
196,101447935,NaN
197,101449373,NaN
198,101449505,NaN


### Исключаем предсказания групп, которых не будет в марте-апреле

### Сохраняем test.csv